In [ ]:
# -*- coding:utf-8 -*-
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from pyowm import OWM
import datetime as dt
import pandas as pd
import calendar

In [ ]:
#파일 경로
save_path = 'c:/cr/MU/'
driver_path = 'c:/cd/chromedriver'

#파일 형식
save_type = 'csv'

#시간정보
today = dt.datetime.today()
yesterday = today - dt.timedelta(days=1)
N_yesterday = yesterday.strftime('%Y-%m-%d')
G_yesterday = yesterday.strftime('%Y%m%d')
today = today.strftime('%y%m%d')

#날씨 체크용
API_Key = '8221f7effc33ce6c0b0626a4a9cc90e0'
City_ID = 1835848 #서울

#검색어 체크용      
naver_url = 'https://datalab.naver.com/keyword/realtimeList.naver?datetime={}T23:00:00'.format(N_yesterday)
google_url = 'https://trends.google.com/trends/hottrends#pn=p23' #대한민국 p=23, 미국 p=1
user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) " +\
"AppleWebKit/537.36 (KHTML, like Gecko) " + \
"Chrome/37.0.2062.94 Safari/537.36"
headers = {"User-Agent": user_agent}

In [3]:
browser = webdriver.Chrome(driver_path)
browser.implicitly_wait(3)

In [4]:
def W_check():
    owm = OWM(API_Key, language='kr')
    base = owm.weather_at_id(City_ID)
    # owm.get_language()

    L = base.get_location()
    City_name = L.get_name()

    W = base.get_weather()
    Temp = W.get_temperature(unit = 'celsius')
    Status = W.get_status()
    
    print('일자: ' + today, City_name, Status, '현재기온: ' + str(Temp['temp']), '최고기온: ' + str(Temp['temp_max']), '최저기온: ' + str(Temp['temp_min']), sep = ' / ')
    
    weather = [today, City_name, Status, str(Temp['temp']), str(Temp['temp_max']), str(Temp['temp_min'])]
    return weather

In [5]:
def N_check():
    N_html = requests.get(naver_url, headers = headers).text
    N_source = bs(N_html, 'html.parser')
    N_ranks = N_source.select('div.keyword_rank.select_date > div > div > ul > li > a > span')
    N_hots = []
    print('- 네이버 인기 검색어 -')
    for i, rank in enumerate(N_ranks):
        N_hots.append(rank.text)
        print(i+1, rank.text)
    return N_hots

In [6]:
def G_check():
    browser.get(google_url)
    G_ranks = browser.find_element_by_id(G_yesterday).text
    G_full = G_ranks.split('\n')
    G_hots = []
    print('- 구글 인기 검색어 -')
    i = 0
    for sentence in G_full:
        hot = G_full.index(sentence) + 3
        if hot % 6 == 0:
            i += 1
            G_hots.append(sentence)
            print(i, sentence)

    return G_hots

In [14]:
def save():
    if save_type == 'txt':
        txt_path = save_path + '{}_MorningUpdate.txt'.format(today)
        with open(txt_path, 'w+', encoding = 'utf8')as f:
            f.write(','.join(weather))
            f.write('\n')
            f.write(','.join(N_hots))
            f.write('\n')
            f.write(','.join(G_hots))
    
    elif save_type == 'csv':
        csv_path = save_path + '{}_MorningUpdate.csv'.format(today)
        with open(csv_path, 'w+', encoding = 'cp949')as f:
            f.write(','.join(weather))
            f.write('\n')
            f.write(','.join(N_hots))
            f.write('\n')
            f.write(','.join(G_hots))
    
#     if save_type == 'csv':
#         csv_path = save_path + '{}_MorningUpdate.csv'.format(today)
#         f = open(csv_path, 'w+', encoding = 'utf8')
#         f.write('\n'.join(map(str, results)))
#         f.close()
    
#     elif save_type == 'txt':
#         txt_path = save_path + '{}_MorningUpdate.txt'.format(today)
#         f = open(txt_path, 'w+', encoding = 'utf8')
#         f.write('\n'.join(map(str, results)))
#         f.close()
    
    print('저장이 완료되었습니다.')

#현재 result를 그대로 저장시 저장이 안됨
#map을 추가하면 txt로는 저장되나 엑셀은 깨짐
#map을 추가하면 리스트의 괄호가 그대로 저장됨
##한줄씩 불러내서 쓰는 방법, 혹은 다른 좋은 방법이 있나 찾아볼것

In [8]:
weather = W_check()
N_hots = N_check()
G_hots = G_check()
results = [weather, N_hots, G_hots]
save()

일자: 180212 / Seoul / Clouds / 현재기온: -11.44 / 최고기온: -9.0 / 최저기온: -14.0
- 네이버 인기 검색어 -
1 복면가왕 집시여인
2 민유라
3 이승훈
4 펠릭스 로흐
5 최다빈
6 효리네민박2
7 선우정아
8 밥상 차리는 남자
9 내보험찾아줌
10 슈가맨
11 서현
12 박보검
13 런닝맨
14 얼굴바꾸기 어플
15 미운우리새끼
16 포항지진
17 쓰리 빌보드
18 아이비
19 화유기
20 이명행
- 구글 인기 검색어 -
1 민유라
2 지진
3 숨은보험금찾기
4 김일성 가면
5 최다빈
6 이승훈
7 793 회 로또 당첨 번호
8 NBC
9 아이스하키
10 바이애슬론
11 루지
저장이 완료되었습니다.


In [15]:
save()

저장이 완료되었습니다.
